In [ ]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive
import ee
ee.Authenticate()
import os

drive.mount('/content/drive') #drive connection



Mounted at /content/drive


In [ ]:
def analyze_built_area(polygon_coords, output_path):


    try:
        ee.Initialize(project='nth-infusion-437410-b6')
    except Exception:
        print("Authentication error. Attempting to authenticate...")
        try:
            ee.Authenticate()
            ee.Initialize(project='nth-infusion-437410-b6')
            print("Authentication successful!")
        except Exception as auth_error:
            print(f"Authentication failed: {auth_error}")
            print("Please run 'earthengine authenticate' in your command line first.")
            return None


    area = ee.Geometry.Polygon([polygon_coords])


    total_area = ee.Number(area.area()).divide(1e6).getInfo()
    print(f'Total Area (km²): {total_area}')


    start_year = 2015
    end_year = 2025
    date_ranges = []

    for year in range(start_year, end_year + 1):

        date_ranges.append({
            'startDate': ee.Date.fromYMD(year, 1, 1),
            'endDate': ee.Date.fromYMD(year, 6, 30),
            'year': year,
            'period': 1
        })


        date_ranges.append({
            'startDate': ee.Date.fromYMD(year, 7, 1),
            'endDate': ee.Date.fromYMD(year, 12, 31),
            'year': year,
            'period': 2
        })


    all_results = []


    for range_info in date_ranges:

        collection = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1') \
            .filterDate(range_info['startDate'], range_info['endDate']) \
            .filterBounds(area)


        count = collection.size().getInfo()
        if count == 0:
            print(f"{range_info['year']} yılı {range_info['period']}. yarıyıl için görüntü bulunamadı.")
            continue

        print(f"{range_info['year']} yılı {range_info['period']}. yarıyıl için kullanılan görüntü sayısı: {count}")


        built_images = collection.select('built')


        image_list = built_images.toList(count)

        for i in range(count):
            image = ee.Image(image_list.get(i))
            image_id = image.id().getInfo()


            built_area_dict = image.multiply(ee.Image.pixelArea()) \
                .reduceRegion(
                    reducer=ee.Reducer.sum(),
                    geometry=area,
                    scale=10,
                    maxPixels=1e10
                ).getInfo()


            built_area_km2 = float(built_area_dict['built']) / 1e6
            built_percentage = (built_area_km2 / total_area) * 100


            result = {
                'year': range_info['year'],
                'period': range_info['period'],
                'date': f"{range_info['year']}-P{range_info['period']}",
                'image_id': image_id,
                'built_area_km2': built_area_km2,
                'built_percentage': built_percentage,
                'total_area_km2': total_area
            }

            all_results.append(result)


    if all_results:
        df = pd.DataFrame(all_results)
        df.to_csv(output_path, index=False)
        print(f"Analysis complete. Results saved to {output_path}")
        return df
    else:
        print("No results found for the given time periods.")
        return None

In [ ]:
def ilce_bazli_arama(izmir_poligon_path , ilce ):
  #ikisi de string formatında olacak. 'izmir poligonun dosyasının colabde bulunabiliyor olması gerek.
  #ilce içinde dosyada nasıl kayıtlıysa o şekilde örneğin 'torbali' ya da 'bornova' gibi girilmesi yeterlidir.
  folder_list = [e for e in os.listdir(polygons_path) if e.find('.') == -1] #path üzerine kayıtlı olan tüm ilçelerin isimleri.
  zone_dict = {} #zone bazlı klasörler.
  allzonedirs = {} #directory'ler direkt burada. İlçe bazlı ayırdım. Üsttekini kullanmaya gerek yok.
  for e in folder_list:
    zones = os.listdir(polygons_path + '/' + e)
    zone_dict[e] = zones
    n = 105 - len(zone_dict[e])
    zone_dict[e] = zone_dict[e] + n * [None]
    allzonedirs[e] =  [polygons_path + '/' + e +'/' + zone for zone in zones]

  for dirs in allzonedirs[ilce]:
    with open(dirs , 'r') as doc :
      kazım = doc.read()
      kerem = re.findall('\d{2}.\d*,\d{2}.\d*', kazım)
      mükremin = []
      for e in kerem :
        mükremin.append(np.array(e.split(',')).astype(float))
    analyze_built_area([list(e) for e in mükremin] , dirs[:-4] + '_rawdata.csv')

In [ ]:
polygons_path = "/content/drive/MyDrive/torbali/izmir_poligonlar" # drive'ından izmir_poligonlar klasörü neredeyse orası.
ilce = 'torbali' #örnek ilçe
ilce_bazli_arama(polygons_path , ilce)